In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

# Load data, crop, and resize

In [ ]:
def load_data(image_folder='data/cat_v1'):

    target_size = 512

    data = []

    for filename in os.listdir(image_folder):
        image_path = os.path.join(image_folder, filename)
        image = Image.open(image_path)
        width, height = image.size
        shorter_side = min(width, height)
        left = (width - shorter_side) // 2
        top = (height - shorter_side) // 2
        right = (width + shorter_side) // 2
        bottom = (height + shorter_side) // 2
        cropped_image = image.crop((left, top, right, bottom))
        resized_image = cropped_image.resize((target_size, target_size))
        image_array = np.array(resized_image)
        if len(image_array.shape) < 3 or image_array.shape[-1] != 3:
            continue
        data.append(image_array[np.newaxis, :, :, :])

    data = np.concatenate(data, axis=0)
    return data

# define generator

In [ ]:
def build_generator(latent_dim):
    model = keras.Sequential()
    model.add(layers.Dense(4 * 4 * 256, input_dim=latent_dim))
    model.add(layers.Reshape((4, 4, 256)))
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(32, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(16, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(8, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(8, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding="same", activation="sigmoid"))
    return model

# define discriminator

In [ ]:
def build_discriminator():
    model = keras.Sequential()
    model.add(layers.Conv2D(8, kernel_size=4, strides=2, padding="same", input_shape=(512, 512, 3)))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(16, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(32, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(512, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="sigmoid"))
    return model

# initilization

In [ ]:
latent_dim = 1024
generator = build_generator(latent_dim)
discriminator = build_discriminator()

generator_optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.2)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.2)

cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

# define loss functions

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# function to show generated images

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, :])
        plt.axis('off')

    plt.savefig(f"saved/cat_v1/generated_images_epoch_{epoch + 1}.png")
    plt.show()
    plt.clf()

# function to train the DCGAN

In [ ]:
@tf.function
def train_step(images):

    noise = tf.random.normal([batch_size, latent_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Training

In [ ]:
batch_size = 128

X_train = load_data()
X_train = (X_train.astype('float32')) / 255
train_dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(X_train.shape[0]).batch(batch_size)


epochs = 5000
for epoch in range(epochs):
    for image_batch in train_dataset:
        train_step(image_batch)

    if (epoch + 1) % 10 == 0:
        generate_and_save_images(generator, epoch, tf.random.normal([16, latent_dim]))
        print(f"Epoch {epoch + 1} completed.")